In [35]:
# Successfully installed ftd2xx-1.3.5 install-1.3.5 --> worked
# Then issue with libftd2xx.so: cannot open shared object file: No such file or directory

"""/:  Bus 02.Port 1: Dev 1, Class=root_hub, Driver=xhci_hcd/11p, 480M
    |__ Port 1: Dev 2, If 0, Class=Vendor Specific Class, Driver=ftdi_sio, 480M
    |__ Port 1: Dev 2, If 1, Class=Vendor Specific Class, Driver=ftdi_sio, 480M"""

"""echo -n 2-1:1.0 | sudo tee /sys/bus/usb/drivers/ftdi_sio/unbind"""

'echo -n 2-1:1.0 | sudo tee /sys/bus/usb/drivers/ftdi_sio/unbind'

In [188]:
import sys
import random
from time import sleep
BLOCK_LEN = 512

In [189]:
import ftd2xx as ft
devs = ft.listDevices()
devs

[b'ibIeBkI8A', b'ibIeBkI8B']

In [190]:
n = ft.createDeviceInfoList()

for index in range(0, n):
    device_info = ft.getDeviceInfoDetail(devnum = index, update = False)
    print(device_info)

{'index': 0, 'flags': 2, 'type': 6, 'id': 67330064, 'location': 8241, 'serial': b'ibIeBkI8A', 'description': b'iCEBreaker V1.0e A', 'handle': <ftd2xx._ftd2xx_linux.LP_c_ulong object at 0x7f034865c140>}
{'index': 1, 'flags': 2, 'type': 6, 'id': 67330064, 'location': 8242, 'serial': b'ibIeBkI8B', 'description': b'iCEBreaker V1.0e B', 'handle': <ftd2xx._ftd2xx_linux.LP_c_ulong object at 0x7f0348621040>}


In [268]:
 d = ft.open(dev = 0)

In [269]:
print(d.getDeviceInfo())
d.setBitMode(0xFF, 0x00)               # Reset mode
sleep(.01)

{'type': 6, 'id': 67330064, 'description': b'iCEBreaker V1.0e A', 'serial': b'ibIeBkI8A'}


In [270]:
sleep(.01)
d.setBitMode(0xFF,0x1)               # Sync FIFO mode - see https://www.ftdichip.com/Support/Knowledgebase/index.html?ft_setbitmode.htm
sleep(.01)
d.setLatencyTimer(2)                   # Receive buffer timeout in msec
sleep(.01)
d.setUSBParameters(2048, 2048)         # Set transfer in/out size to 4096
sleep(.01)
d.setFlowControl(ft.defines.FLOW_RTS_CTS, 0, 0)   # Flow control method
sleep(.01)

* I've implemented Async FIFO Loopback.
* Write 0x7X - 1byte. (7:4 bits opcode = 7=LOOPBACK; 3:0 bits data)
* Then read 1byte, written byte should come back.

In [271]:
wr = d.write(str(0x70))
B  = d.write(str(0x7))

In [272]:
rx_data = d.read(1)
rx_data.decode('utf-8')

'7'

In [267]:
d.close()